In [16]:
import geopandas as gpd
import pandas as pd

## Open State data

In [17]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)
counties_gdf = counties_gdf[counties_gdf["STATEFP"] != "09"][["geometry", "AFFGEOID"]]
counties_gdf = counties_gdf.to_crs(4326)

#### Format for CT

In [18]:
ct_gdf = gpd.read_file("data/CT_Planning_Regions.geojson")
ct_gdf["AFFGEOID"] = "0500000US" + ct_gdf["PlanningRegionFIPS_GEOID"]

In [19]:
counties_gdf = pd.concat(
    [ct_gdf[["geometry", "AFFGEOID"]], counties_gdf], ignore_index=True
)

## Get Disability Percents

In [20]:
disability_df = pd.read_csv(
    "data/ACSST5Y2023.S1810_2025-05-07T160024/ACSST5Y2023.S1810-Data.csv",
    low_memory=False,
)

#### Get Column names

In [21]:
acs_column_names = pd.read_csv(
    "data/ACSST5Y2023.S1810_2025-05-07T160024/ACSST5Y2023.S1810-Column-Metadata.csv"
)

In [22]:
disability_cols = list(
    acs_column_names[
        (acs_column_names["Column Name"] == "S1810_C01_001E")
        | (acs_column_names["Column Name"] == "S1810_C03_001E")
    ]["Column Name"]
)

#### Back to Data

In [23]:
disability_df = acs_internet[["GEO_ID", *disability_cols]]W
disability_df.columns = disability_df.iloc[0]
disability_df = disability_df[1:]
disability_df = disability_df.rename(
    columns={
        "Estimate!!Total!!Total civilian noninstitutionalized population": "TOTAL",
        "Estimate!!Percent with a disability!!Total civilian noninstitutionalized population": "PERCENT",
        "Geography": "AFFGEOID",
    }
)

## Merge Data

In [67]:
disability_gdf = counties_gdf.merge(disability_df, on="AFFGEOID", how="left")

In [68]:
disability_gdf["county_id"] = disability_gdf["AFFGEOID"].str[9:]
disability_gdf["state_id"] = disability_gdf["AFFGEOID"].str[9:11]

In [69]:
disability_gdf = disability_gdf.fillna(0)
disability_gdf["TOTAL"] = disability_gdf["TOTAL"].astype(int)
disability_gdf["PERCENT"] = disability_gdf["PERCENT"].astype(float)

In [70]:
disability_gdf = disability_gdf[
    ~disability_gdf["state_id"].isin(["60", "66", "69", "72", "78"])
].reset_index(drop=True)

In [71]:
disability_gdf = disability_gdf.to_crs(9311)
disability_gdf.to_file("data/disability.gpkg")

In [72]:
disability_gdf.sort_values("PERCENT", ascending=False)

,geometry,AFFGEOID,TOTAL,PERCENT,county_id,state_id
216,"MULTIPOLYGON (((261305.708 -2003968.269, 26196...",0500000US48261,52,51.9,48261,48
804,"POLYGON ((-377925.058 -1434337.126, -377709.86...",0500000US48301,54,42.6,48301,48
1724,"POLYGON ((-843773.732 -1259482.941, -842510.80...",0500000US35003,3679,39.7,35003,35
2313,"POLYGON ((1042893.659 -817572.517, 1044303.914...",0500000US21143,7314,38.0,21143,21
305,"POLYGON ((1534921.728 -705507.468, 1535186.303...",0500000US51051,13466,37.7,51051,51
...,...,...,...,...,...,...
1052,"POLYGON ((-870715.457 22385.368, -847028.453 1...",0500000US56039,23276,6.4,56039,56
649,"POLYGON ((-791897.096 -735282.839, -791872.081...",0500000US08113,8013,6.0,08113,08
301,"POLYGON ((-977362.379 -400353.839, -977101.463...",0500000US49043,42612,5.9,49043,49
1012,"POLYGON ((-701862.731 -779501.501, -701541.255...",0500000US08111,690,2.9,08111,08
